In [ ]:
!pip install llama-index
!pip install slack_sdk

In [ ]:
import os

from dotenv import find_dotenv, load_dotenv
from llama_index import GPTListIndex, SlackReader

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

load_dotenv()
load_dotenv(dotenv_path=find_dotenv(filename=".env.local"), override=True)


In [ ]:
from llama_index import LLMPredictor, ServiceContext, PromptHelper
from langchain.chat_models import ChatOpenAI

llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"))
prompt_helper = PromptHelper.from_llm_predictor(llm_predictor=llm_predictor)
service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor,
    chunk_size_limit=1024,
    prompt_helper=prompt_helper,
)

In [ ]:
import datetime
today = datetime.datetime.today()
last_month = today - datetime.timedelta(days=7)

slack_token = os.getenv("SLACK_API_KEY")
# channel_ids = ["dev", "hash-app", "hash-graph", "help", "general", "product", "ai-ml"]
channel_ids = ["C02TWBTT3ED", "C022217GAHF", "C03F7V6DU9M", "C02K2ARC1BK", "CM3JJMYRL", "C038SGEKHR6", "C052SV8JF24"]
documents = SlackReader(slack_token=slack_token, earliest_date=last_month).load_data(channel_ids=channel_ids)
index = GPTListIndex.from_documents(documents, service_context=service_context)

In [ ]:
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query("What is tim doing")

In [ ]:
response